In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm

# Build EPTS Gaussian Distribution

In [3]:
#using the following EPTS factor distributions from the 2019 SRTR:
#age: [8.3% 18-34, 24% 35-49, 43.5% 50-64, 24.2% > 64]
#diabeties prevalence: [46.8% yes, 53.2% no]
#time on dialysis: [18.2% none, 11.6% <1, 21.3% 1-3, 16.7% 3-5, 32.2% >5]
#transplant history: [88.2% first, 11.8% retransplant]

#compute mean vectors
binned_age_midpoints = np.array([26, 42, 57, 72])
age_probabilites = np.array([0.083, 0.240, 0.435, 0.242])
mean_age = np.sum(binned_age_midpoints * age_probabilites)

binned_dialysis_midpoints = np.array([0, 0.5, 2, 4, 6])
dialysis_probabilites = np.array([0.182, 0.116, 0.213, 0.167, 0.322])
mean_dialysis = np.sum(binned_dialysis_midpoints * dialysis_probabilites)

mean_diabetes_latent = 0.468
mean_prior_latent = 0.118

mu = np.array([mean_age, mean_dialysis, mean_diabetes_latent, mean_prior_latent])
print("Mean vector:\n", mu)

#estimate standard deviations
var_age = np.sum(age_probabilites * (binned_age_midpoints - mean_age)**2)
sd_age = np.sqrt(var_age)

var_dial = np.sum(dialysis_probabilites * (binned_dialysis_midpoints - mean_dialysis)**2)
sd_dial = np.sqrt(var_dial)

#for the bernoullis, we can't use the SD directly in the Gaussian because binary variables have large variances that would overwhelm the covariant structure
#take some controlled fraction of the bernoulli SDs (~35-40%) here i use 47%
var_diab = mean_diabetes_latent * (1 - mean_diabetes_latent)
sd_diab = np.sqrt(var_diab)
sd_diab_latent = 0.37*sd_diab

var_prior = mean_prior_latent * (1 - mean_prior_latent)
sd_prior = np.sqrt(var_prior)
sd_prior_latent = 0.37*sd_prior

sd = np.array([sd_age, sd_dial, sd_diab_latent, sd_prior_latent])
print("SD vector:\n", sd)

#estimate correlations
rho_age_diab = 0.25
rho_age_dial = 0.20
rho_diab_dial = 0.30

rho_age_prior = 0.10
rho_dial_prior = 0.15
rho_diab_prior = 0.10

#build covariance matrix
cov = np.array([
    [sd_age**2,
     rho_age_dial * sd_age * sd_dial,
     rho_age_diab * sd_age * sd_diab_latent,
     rho_age_prior * sd_age * sd_prior_latent],

    [rho_age_dial * sd_age * sd_dial,
     sd_dial**2,
     rho_diab_dial * sd_dial * sd_diab_latent,
     rho_dial_prior * sd_dial * sd_prior_latent],

    [rho_age_diab * sd_age * sd_diab_latent,
     rho_diab_dial * sd_dial * sd_diab_latent,
     sd_diab_latent**2,
     rho_diab_prior * sd_diab_latent * sd_prior_latent],

    [rho_age_prior * sd_age * sd_prior_latent,
     rho_dial_prior * sd_dial * sd_prior_latent,
     rho_diab_prior * sd_diab_latent * sd_prior_latent,
     sd_prior_latent**2]
])

print("\nCovariance matrix:\n", cov)

Mean vector:
 [54.457  3.084  0.468  0.118]
SD vector:
 [13.48132601  2.37359306  0.18462073  0.119365  ]

Covariance matrix:
 [[1.81746151e+02 6.39983636e+00 6.22233067e-01 1.60919855e-01]
 [6.39983636e+00 5.63394400e+00 1.31464346e-01 4.24985920e-02]
 [6.22233067e-01 1.31464346e-01 3.40848144e-02 2.20372545e-03]
 [1.60919855e-01 4.24985920e-02 2.20372545e-03 1.42480044e-02]]


In [4]:
N = 150000
rng = np.random.default_rng(0)
samples = rng.multivariate_normal(mu, cov, size=N)

Age = samples[:, 0]
DialysisYears = samples[:, 1]
DiabetesLatent = samples[:, 2]
PriorTxLatent = samples[:, 3]

#truncate dialysis years at 0, age at 18
DialysisYears = np.clip(DialysisYears, 0, None)
Age = np.clip(Age, 18, None)

#thresholds based on target SRTR prevalences
t_diab = norm.ppf(1 - 0.468, loc=mean_diabetes_latent, scale=sd_diab_latent)
t_prior = norm.ppf(1 - 0.118, loc=mean_prior_latent, scale=sd_prior_latent)

#apply thresholds to get binary values from the distribution
Diabetes = (DiabetesLatent > t_diab).astype(int)
PriorTx = (PriorTxLatent > t_prior).astype(int)

df = pd.DataFrame({
    "Age": Age,
    "DialysisYears": DialysisYears,
    "Diabetes": Diabetes,
    "PriorTx": PriorTx
})

# Add Raw EPTS and EPTS Score

In [5]:
#calculate raw EPTS score
age_component = np.maximum(df["Age"] - 25, 0)
log_dialysis = np.log(df["DialysisYears"] + 1)
indicator_no_dialysis = (df["DialysisYears"] == 0).astype(int)

raw_EPTS = (
    0.047 * age_component
    - 0.015 * df["Diabetes"] * age_component
    + 0.398 * df["PriorTx"]
    - 0.237 * df["Diabetes"] * df["PriorTx"]
    + 0.315 * log_dialysis
    - 0.099 * df["Diabetes"] * log_dialysis
    + 0.130 * indicator_no_dialysis
    - 0.348 * df["Diabetes"] * indicator_no_dialysis
    + 1.262 * df["Diabetes"]
)

df["RawEPTS"] = raw_EPTS

#get EPTS score from the official EPTS mapping table

epts_upper_bounds = [
    0.01184827379332, 0.21310210737676, 0.40252326481930, 0.52800000000000,
    0.61801976010923, 0.71057871011496, 0.78782781442424, 0.85771064385749,
    0.92387462259398, 0.98764264652321, 1.04590485968515, 1.09933073333614,
    1.15638329911020, 1.20971321013005, 1.25784543260537, 1.30468387455511,
    1.35114305270363, 1.39696372347707, 1.44043463381246, 1.48161190965092,
    1.52021149897331, 1.55920533880904, 1.59484531143053, 1.62808213552361,
    1.65982819986311, 1.69011088295688, 1.72058765890643, 1.74844920196828,
    1.77551471594798, 1.80172896022488, 1.82626695257618, 1.85104928131417,
    1.87446885694730, 1.89736960985626, 1.92001468736874, 1.94089972738606,
    1.96259067534531, 1.98299520876112, 2.00301505817933, 2.02300693973232,
    2.04283275957921, 2.06033324099255, 2.07901711156742, 2.09753586134351,
    2.11445516769336, 2.13195550992471, 2.14932717316906, 2.16505133470226,
    2.18131759069131, 2.19695261029016, 2.21262422997947, 2.22883854789024,
    2.24526417931112, 2.26074101528416, 2.27693086926763, 2.29203216974675,
    2.30747364818617, 2.32239630390144, 2.33719849418207, 2.35233619399769,
    2.36720658011572, 2.38223956194387, 2.39716290212183, 2.41171798708993,
    2.42672344324962, 2.44241981474590, 2.45704382237689, 2.47204106776181,
    2.48615153831781, 2.50130321697467, 2.51610951403148, 2.53117864476386,
    2.54691372810912, 2.56188750397993, 2.57783580816370, 2.59371564589885,
    2.60883983572895, 2.62490395223486, 2.64215263518138, 2.65902464065708,
    2.67751421968340, 2.69582272416153, 2.71585259723488, 2.73434907597536,
    2.75311920676514, 2.77272279260780, 2.79350504661888, 2.81476309520569,
    2.83763973196257, 2.85957991275097, 2.88260648450195, 2.90694476081954,
    2.93331208731685, 2.96201988546123, 2.99154579542839, 3.02388623085550,
    3.06016380052332, 3.10215168515328, 3.15286397325764, 3.23640591422491,
    3.76844960318675, 999999999.0
]

epts_percentiles = np.arange(0, 101)  #0% to 100%

#map raw EPTS to percentile score using table

def map_raw_epts_to_score(raw_epts):
    idx = np.searchsorted(epts_upper_bounds, raw_epts, side="right") - 1
    idx = np.clip(idx, 0, 100)
    return epts_percentiles[idx]

df["EPTSScore"] = df["RawEPTS"].apply(map_raw_epts_to_score)

print(df.head(10))

         Age  DialysisYears  Diabetes  PriorTx   RawEPTS  EPTSScore
0  52.751160       3.331531         0        0  1.766070         27
1  61.709498       2.510279         0        0  2.120890         44
2  63.836907       6.365154         1        0  2.936081         92
3  85.781854       4.725796         1        0  3.583935         99
4  61.767604       4.085717         0        0  2.240405         51
5  56.304879       0.000000         1        0  2.045756         40
6  42.284906       2.421550         1        0  2.080817         42
7  60.645766       2.793059         1        0  2.690630         80
8  56.649290       1.906261         0        0  1.823580         29
9  63.260639       3.706757         0        1  2.684185         80


# Add Categorical Variables Per Patient

In [6]:
def sample_categorical(labels, probs, size, rng):
    labels = np.asarray(labels)
    probs = np.asarray(probs, dtype=float)
    probs /= probs.sum()
    idx = rng.choice(len(labels), size=size, p=probs)
    return labels[idx]

N = len(df)
rng = np.random.default_rng(1)

#sample sex
sex_labels = ["F", "M"]
sex_probs = [0.381, 0.619]

df["Sex"] = sample_categorical(sex_labels, sex_probs, N, rng)

#sample ethnicity
eth_labels = ["White", "Black", "Hispanic", "Asian", "Other"]
eth_probs = [0.355, 0.323, 0.207, 0.097, 0.018]

df["Ethnicity"] = sample_categorical(eth_labels, eth_probs, N, rng)

#sample distance to treatment center (miles)
dist_labels = ["<50", "50-100", "100-150", "150-250", ">250"]
dist_probs  = [0.663, 0.157, 0.069, 0.061, 0.047]

df["DistancetoCenterMiles"] = sample_categorical(dist_labels, dist_probs, N, rng)

#sample blood type
bt_labels = ["A", "B", "AB", "O"]
bt_probs  = [0.273, 0.167, 0.025, 0.535]

df["BloodType"] = sample_categorical(bt_labels, bt_probs, N, rng)

#sample weight time (in years)
wait_labels = ["<1", "1-2", "2-3", "3-4", "4-5", ">5"]
wait_probs  = [0.329, 0.215, 0.143, 0.104, 0.072, 0.137]

df["WaitTimeYears"] = sample_categorical(wait_labels, wait_probs, N, rng)

df.head()

,Age,DialysisYears,Diabetes,PriorTx,RawEPTS,EPTSScore,Sex,Ethnicity,DistancetoCenterMiles,BloodType,WaitTimeYears
0,52.751160,3.331531,0,0,1.766070,27,M,Hispanic,<50,O,3-4
1,61.709498,2.510279,0,0,2.120890,44,M,Black,50-100,A,1-2
2,63.836907,6.365154,1,0,2.936081,92,F,White,<50,O,2-3
3,85.781854,4.725796,1,0,3.583935,99,M,Black,<50,O,<1
4,61.767604,4.085717,0,0,2.240405,51,F,Hispanic,<50,O,2-3


# Simulate Donor List

In [7]:
N_donors = 20000
rng = np.random.default_rng(2)

#get approximation of KDPI continous values from binned values
kdpi_bins = [(0,20), (20,35), (35,85), (85,100)]
kdpi_probs = np.array([0.206, 0.111, 0.447, 0.186]) #have to renormalize since there are 5% "unknown KDPI"
kdpi_probs = kdpi_probs / kdpi_probs.sum()

#choose bins
bin_idx = rng.choice(len(kdpi_bins), size=N_donors, p=kdpi_probs)

KDPI_continuous = np.zeros(N_donors)

for i, idx in enumerate(bin_idx):
    low, high = kdpi_bins[idx]
    KDPI_continuous[i] = rng.uniform(low, high)

#donor blood type distribution
donor_bt_labels = ["A", "B", "AB", "O"]
donor_bt_probs  = [0.413, 0.128, 0.054, 0.405]

DonorBloodType = sample_categorical(donor_bt_labels, donor_bt_probs, N_donors, rng)

age_bins = [(18, 34), (35, 49), (50, 64), (65, 90)]
age_probs = np.array([0.250, 0.263, 0.288, 0.086])
age_probs = age_probs / age_probs.sum()

#choose an age bin for each donor
bin_idx_age = rng.choice(len(age_bins), size=N_donors, p=age_probs)

#sample a continuous donor age
DonorAge = np.zeros(N_donors)

for i, idx in enumerate(bin_idx_age):
    low, high = age_bins[idx]
    DonorAge[i] = rng.uniform(low, high)


#combine into donor dataframe
donors = pd.DataFrame({
    "KDPI": KDPI_continuous,
    "DonorBloodType": DonorBloodType,
    "DonorAge": DonorAge
})

donors.head(20)

,KDPI,DonorBloodType,DonorAge
0,22.079660,O,21.321056
1,20.868635,AB,24.983081
2,87.416243,O,63.114895
3,10.743542,A,39.847888
4,76.192908,O,57.201801
5,61.325574,A,55.542667
6,12.032628,A,54.230315
7,7.673953,O,32.051051
8,21.914772,A,33.486498
9,64.314004,O,33.988038


In [ ]:
df.to_csv('patients.csv', index=False)
donors.to_csv('donors.csv', index=False)